In [ ]:
import requests
import pandas as pd
import time

"""
Update Parameters Here
"""
COLLECTION = "quaks"
CONTRACT = "0x07bbdaf30e89ea3ecf6cadc80d6e7c4b0843c729"
ETHER_UNITS = 1e18
LIMIT = 200
MAX_OFFSET = 100000
MAX_TOKEN_ID = 6000

def get_all_sales():
    events = []
    new_events = []
    offset = 0 
    
    while True:
        
        query_string = {
            "asset_contract_address" : CONTRACT,
            "event_type" : "successful",
            "limit" : LIMIT,
            "offset" : offset
        }

        response = requests.request("GET", url = "https://api.opensea.io/api/v1/events" , params = query_string).json()
        
        new_events = response['asset_events']
        events.extend(new_events)
        if len(new_events) < LIMIT or offset > MAX_OFFSET:
            break
            
        offset += LIMIT
        
    return events

In [ ]:
"""
Generate Dataset
"""

events = get_all_sales()
events_list = []
seen = set()

for event in events: 
    event_dict = dict()
    
    try:
        event_dict = {
            "id" : int(event['asset']['token_id']),
            "price" : float(event['total_price']) / ETHER_UNITS,
            "date_sold" : event['transaction']['timestamp']
        }

        events_list.append(event_dict)
        seen.add(int(event['asset']['token_id']))
    except:
        print("error retrieving token data")


data = pd.DataFrame(events_list)
data = data.sort_values(by=['date_sold']).drop_duplicates('id',keep='first')
data.to_csv("{}.csv".format(COLLECTION))